# ルビ振り・かな漢字変換を用いたクエリ拡張
ドキュメントは作成者により、ひらがなを使用したり、漢字を使用したりと、表記揺れが存在します。<br>
そのため、例えば「薔薇」を検索クエリとして使用しても、「ばら」と記述されたドキュメントは検出されず見つけ出すことはできません。

情報検索の領域では、この問題に対処するために、「クエリ拡張」と呼ばれる方法が用いられます。これにより、表記揺れをOR検索します。<br>
例えば、「薔薇」に対しては、「薔薇 OR ばら OR バラ」のようにクエリを拡張することが可能です。

本記事では、テキスト解析 Web API のルビ振りやかな漢字変換を使用し、簡単なクエリ拡張を実現する方法について解説します。

## Pythonからリクエストするメソッドを用意
ルビ振り・かな漢字変換の機能を使うためのメソッドを定義します。
これらを用いてクエリ拡張を行います。

In [1]:
import json
from urllib import request

APPID = 'あなたの Client ID（アプリケーション ID）' 
FURIGANA_URL = "https://jlp.yahooapis.jp/FuriganaService/V2/furigana"
KANAKAN_URL = "https://jlp.yahooapis.jp/JIMService/V2/conversion"


def post_furigana_service(query: str) -> str:
    headers = {
        "Content-Type": "application/json",
        "User-Agent": "Yahoo AppID: {}".format(APPID),
    }
    param_dic = {
        "id": "1234-1",
        "jsonrpc": "2.0",
        "method": "jlp.furiganaservice.furigana",
        "params": {"q": query},
    }
    params = json.dumps(param_dic).encode()
    req = request.Request(FURIGANA_URL, params, headers)
    with request.urlopen(req) as res:
        body = res.read()
    return body.decode()

def post_kanakan_service(query: str) -> str:
    headers = {
        "Content-Type": "application/json",
        "User-Agent": "Yahoo AppID: {}".format(APPID),
    }
    param_dic = {
        "id": "1234-1",
        "jsonrpc": "2.0",
        "method": "jlp.jimservice.conversion",
        "params": {"q": query},
    }
    params = json.dumps(param_dic).encode()
    req = request.Request(KANAKAN_URL, params, headers)
    with request.urlopen(req) as res:
        body = res.read()
    return body.decode()

（※）Client ID については[こちら](../02_API_Specifications/00_Overview.md#client-id%E3%82%A2%E3%83%97%E3%83%AA%E3%82%B1%E3%83%BC%E3%82%B7%E3%83%A7%E3%83%B3id)をご覧ください。

In [3]:
# ルビ振りの結果を確認
q = "今日はいい天気です"
post_furigana_service(q)

'{"id":"1234-1","jsonrpc":"2.0","result":{"word":[{"furigana":"きょう","roman":"kyou","surface":"今日"},{"furigana":"は","roman":"ha","surface":"は"},{"furigana":"いい","roman":"ii","surface":"いい"},{"furigana":"てんき","roman":"tenki","surface":"天気"},{"furigana":"です","roman":"desu","surface":"です"}]}}'

In [4]:
# かな漢字変換の結果を確認
q = "きょうはいいてんきです"
post_kanakan_service(q)

'{"id":"1234-1","jsonrpc":"2.0","result":{"segment":[{"candidate":["今日は","きょうは","教派","興は","京は","強は","経は","教は","境は","凶は","卿は","峡は","協は","香は","恭は","享は"],"reading":"きょうは"},{"candidate":["いい","良い","言い","好い","善い","飯","唯々","謂","伊井","井伊","云い","謂い","遺意"],"reading":"いい"},{"candidate":["天気です","転機です","転記です","転帰です","天機です","てんきです"],"reading":"てんきです"}]}}'

## クエリ拡張を行う
上述のメソッドを使用してテキスト解析 Web API のルビ振りとかな漢字変換にアクセスし、対象クエリの「ひらがな」、「ローマ字」、および「かな漢字候補の一番目」を取得します。<br>
これらはクエリ拡張に使用します。

In [5]:
from typing import List

def expand_query(q: str) -> List[str]:
    # ルビ振りの結果を取得
    res_furigana = json.loads(post_furigana_service(q))
    # ひらがなの結果を取得
    q_hiragana = "".join([ seg["furigana"] if "furigana" in seg else seg["surface"]
                                              for seg in res_furigana["result"]["word"]])
    # ローマ字の結果を取得
    q_roman = "".join([ seg["roman"] if "roman" in seg else seg["surface"] 
                                           for seg in res_furigana["result"]["word"]])
    # ひらがなの結果を用いて、かな漢字変換の結果を取得
    res_kanakan = json.loads(post_kanakan_service(q_hiragana))
    # 今回はかな漢字変換候補の一番目を利用
    q_kanakan = "".join([ seg["candidate"][0] for seg in res_kanakan["result"]["segment"]])
    
    or_words = list(set([q, q_hiragana, q_roman, q_kanakan]))
    return or_words

In [6]:
q = "ブタ肉"
expand_query(q)

['ブタ肉', 'ぶたにく', '豚肉', 'butaniku']

 # 実行例
サンプルのドキュメント集合を用意し、先ほど拡張したクエリで OR 検索を行ってみます。

In [ ]:
!pip3 install pandas

In [7]:
import pandas as pd

documents = pd.DataFrame({
    "text": [
        "豚肉と野菜の炒め",
        "牛肉のすき焼き",
        "ブタ肉の回鍋肉",
        "butaniku shabu shabu",
        "チキンカレー"
    ]
})
documents

,text
0,豚肉と野菜の炒め
1,牛肉のすき焼き
2,ブタ肉の回鍋肉
3,butaniku shabu shabu
4,チキンカレー


### クエリを拡張せずに検索を行った場合

In [8]:
q = "ブタ肉"
pd_query = f'text.str.contains("{q}")' #<列名>.str.contains（<テキスト>）を用いることで<列名>に<テキスト>を含む行を取得できる
print(pd_query) 
documents.query(pd_query)

text.str.contains("ブタ肉")


,text
2,ブタ肉の回鍋肉


### クエリを拡張して検索を行った場合

In [9]:
q = "ブタ肉"
pd_query = ' or '.join([f'text.str.contains("{q_i}")' for q_i in expand_query(q)])
print(pd_query)
documents.query(pd_query)

text.str.contains("ブタ肉") or text.str.contains("ぶたにく") or text.str.contains("豚肉") or text.str.contains("butaniku")


,text
0,豚肉と野菜の炒め
2,ブタ肉の回鍋肉
3,butaniku shabu shabu


元のクエリを使用した場合、関連するドキュメントが1件しか見つからなかったのに対し、クエリを拡張して検索することで、関連するドキュメントを1件から3件に増やすことができました。

本記事では、簡単な例として実装しましたが、複数のタームを考慮に入れたり、かな漢字変換候補の2, 3番目を利用するなど、さまざまな工夫が可能です。<br>
ただし、クエリを過度に拡張すると、関連のないドキュメントも検索結果に含まれてしまう可能性があるため、その点には注意が必要です。

さらに今回使用したルビ振りとかな漢字変換の機能以外にもクエリ拡張に役立つ機能が存在します。これらを活用して、より高度なクエリ拡張を試みてみてください。

## 使用しているテキスト解析 Web API
- [ルビ振り](../02_API_Specifications/03_FuriganaService.md)
- [かな漢字変換](../02_API_Specifications/02_JIMService.md)

## 著者
LINEヤフー株式会社 言語処理エンジニア  
伊奈 拓郎（[@tuna_takenoko](https://x.com/tuna_takenoko)）